# Expectation Maximization

[This pdf](https://github.com/mtomassoli/information-theory-tutorial) helped me a lot, but mainly with other stuff than EM. I also found a great blog [here](https://nipunbatra.github.io/blog/2014/em.html). Also, [this](https://www.cs.utah.edu/~piyush/teaching/EM_algorithm.pdf) explanation is more mathy.

Iterative method, two steps:
* Expectation - Creates a function for the expectation of the log-likelihood evaluated using the current estimate for the parameters. What are calculated in the first step are the fixed, data-dependent parameters of the function Q.
* Maximization - Computes parameters maximizing the expected log-likelihood found on the E step. Once the parameters of Q are known, it is fully determined and is maximized in the second (M) step of an EM algorithm.

Other methods exist to find maximum likelihood estimates, such as gradient descent, conjugate gradient, or variants of the Gauss–Newton algorithm. Unlike EM, such methods typically require the evaluation of first and/or second derivatives of the likelihood function.

1. First, initialize the parameters $\boldsymbol {\theta }$ to some random values.
2. Compute the probability of each possible value of $\mathbf {Z}$, given $\boldsymbol {\theta }$.
3. Then, use the just-computed values of $\mathbf {Z}$  to compute a better estimate for the parameters ${\boldsymbol {\theta }}$.
4. Iterate steps 2 and 3 until convergence.

## Example
We have 0 or 1 sample data drawn from two Bernoulli distributions. Variable with Bernoulli distribution takes 1 with probability p and 0 with probability 1 - p, where p is a distribution parameter. We don't know from which distribution a sample comes from.

Maximum likelihood estimate of p is a sample mean $\frac{1}{n} \sum_i^n x_i$

In [128]:
import functools as ft
import itertools as it
import json
import math
import operator as op
import os

from IPython.display import display
from ipywidgets import interact, interact_manual, widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import misc, stats
from sklearn import metrics

In [129]:
observations = np.array([[1,0,0,0,1,1,0,1,0,1], # the assumption is that each row comes from a single distribution
                         [1,1,1,1,0,1,1,1,1,1], # makes sense, without it the solution would probably be that 
                         [1,0,1,1,1,1,1,0,1,1], # all ones comes from a dist with p=1 and all zeros from p=0
                         [1,0,1,0,0,0,1,1,0,0], 
                         [0,1,1,1,0,1,1,1,0,1]])

In [130]:
# if we know from which distributions the observations come from, we can simply calculate 
# the params from ml estimator

distribution_ids = np.array([0, 1, 1, 0, 1]) # hidden params

p_0 = observations[distribution_ids == 0].mean()
p_1 = observations[distribution_ids == 1].mean()

print(p_0, p_1)

0.45 0.8


In [157]:
# start from some random numbers, you might get different results depending on choice though
# visible: model_n, visible_size; hidden: model_n, hidden_size; 
# observations: observation_n (= hidden_size), observation_size 

visible_params = np.array([[0.6], [0.5]])

def estimate_hidden_params(observations, visible_params):
    hits = observations.sum(axis=1)
    _, observation_size = observations.shape
    
    model_probs = stats.binom.pmf(hits, observation_size, visible_params)
        
    total_probs = model_probs.sum(axis=0)
    return model_probs / total_probs
    
hidden_params = estimate_hidden_params(observations, visible_params)
print(hidden_params)

def maximize_visible_params(observations, hidden_params):
    per_observation_estimates = observations.mean(axis=1)
    # (model_n, hidden_size) x (observation_n = hidden_size, visible_size) = (model_n, visible_size)
    visible_estimates = hidden_params @ per_observation_estimates / hidden_params.sum(axis=1)
    return visible_estimates[:, np.newaxis]
      
visible_params = maximize_visible_params(observations, hidden_params)
print(visible_params)

[[ 0.44914893  0.80498552  0.73346716  0.35215613  0.64721512]
 [ 0.55085107  0.19501448  0.26653284  0.64784387  0.35278488]]
[[ 0.71301224]
 [ 0.58133931]]


In [158]:
def expectation_maximization(observations, initial_visible, iterations):
    visible_params = initial_visible
    for i in range(iterations):
        hidden_params = estimate_hidden_params(observations, visible_params)
        visible_params = maximize_visible_params(observations, hidden_params)
    return visible_params, hidden_params

visible_params, hidden_params = expectation_maximization(
    observations, np.array([[0.6], [0.5]]), 1000
)
print(visible_params)
print(hidden_params)

[[ 0.79678907]
 [ 0.51958312]]
[[ 0.10300871  0.95201348  0.84549373  0.03070315  0.6014986 ]
 [ 0.89699129  0.04798652  0.15450627  0.96929685  0.3985014 ]]


The real values were 

* visible params [0.45 0.8]
* hidden params [0, 1, 1, 0, 1]

The algorithms swapped first distribution with second, but it's perfectly fine, they're just identified in different way.

In [160]:
expectation_maximization(
    np.array([[1], [0], [1], [0], [0], 
              [1], [0], [1], [1], [0], 
              [0], [0], [1], [0], [1], 
              [1], [0], [1], [0], [0]]), 
    np.array([[0.90], [0.10]]), 
    1000
)

(array([[ 0.83938928],
        [ 0.06061072]]),
 array([[ 0.93265475,  0.14600975,  0.93265475,  0.14600975,  0.14600975,
          0.93265475,  0.14600975,  0.93265475,  0.93265475,  0.14600975,
          0.14600975,  0.14600975,  0.93265475,  0.14600975,  0.93265475,
          0.93265475,  0.14600975,  0.93265475,  0.14600975,  0.14600975],
        [ 0.06734525,  0.85399025,  0.06734525,  0.85399025,  0.85399025,
          0.06734525,  0.85399025,  0.06734525,  0.06734525,  0.85399025,
          0.85399025,  0.85399025,  0.06734525,  0.85399025,  0.06734525,
          0.06734525,  0.85399025,  0.06734525,  0.85399025,  0.85399025]]))

In [161]:
expectation_maximization(
    np.array([[1, 0, 1, 0, 0, 
               1, 0, 1, 1, 0, 
               0, 0, 1, 0, 1, 
               1, 0, 1, 0, 0]]), 
    np.array([[0.75], [0.25]]), 
    1000
)

(array([[ 0.45],
        [ 0.45]]), array([[ 0.5],
        [ 0.5]]))

In [136]:
expectation_maximization(
    np.array([[1, 0, 1, 0, 0, 
               1, 0, 1, 1, 0, 
               0, 0, 1, 0, 1, 
               1, 0, 1, 0, 0]]), 
    np.array([[0.75]]), 
    1000
)

(array([ 0.45]), array([[ 1.]]))

In [137]:
expectation_maximization(
    np.array([[1, 0, 1, 1, 1], 
              [1, 0, 1, 1, 0], 
              [0, 0, 1, 0, 1], 
              [1, 0, 0, 0, 0]]), 
    np.array([[0.8], [0.6], [0.4], [0.2]]), 
    10000
)

(array([ 0.50237364,  0.50217056,  0.49782944,  0.49762636]),
 array([[ 0.25356136,  0.25118595,  0.2488122 ,  0.24644049],
        [ 0.2532549 ,  0.25108626,  0.24891559,  0.24674325],
        [ 0.24674325,  0.24891559,  0.25108626,  0.2532549 ],
        [ 0.24644049,  0.2488122 ,  0.25118595,  0.25356136]]))

# Gaussian Mixtures

I will now use the same algorithm, but for continuous-valued observations from normal distributions.

https://www.ics.uci.edu/~smyth/courses/cs274/notes/EMnotes.pdf (but it's really similar to the solution above)

Important observation: When setting initial parameters, you need to use high std deviations, so that the algorithms gets a non-zero probability for all observations. If you set the stdev to something very low, the probability of some observations may be 0 for all models. In theory they should be never exactly 0, but they will be too low to correctly represent in float64 datatype. When you get all 0s further calculations will result in a lot of NaNs and algorithm will fail to produce a result.

In [233]:
def estimate_hidden_params_gm(observations, visible_params):
    model_n = visible_params.shape[0]
    observation_n = observations.shape[0]
    model_probs = np.zeros((model_n, observation_n), dtype=np.float64)
    
    for i, visible_params_row in enumerate(visible_params):
        single_model_probs = stats.norm.pdf(observations, *visible_params_row)
        single_model_per_row_probs = np.product(single_model_probs, axis=1)
        model_probs[i, :] = single_model_per_row_probs
    
    total_probs = model_probs.sum(axis=0)
    return model_probs / total_probs

def maximize_visible_params_gm(observations, hidden_params):
    mean_estimates = observations.mean(axis=1)
    std_estimates = observations.std(axis=1, ddof=1)
    per_observation_estimates = np.vstack((mean_estimates, std_estimates)).T
    # (model_n, hidden_size) x (observation_n = hidden_size, visible_size) = (model_n, visible_size)
    visible_estimates = hidden_params @ per_observation_estimates / hidden_params.sum(axis=1)
    return visible_estimates

def expectation_maximization_gm(observations, initial_visible, iterations):
    visible_params = initial_visible
    for i in range(iterations):
        hidden_params = estimate_hidden_params_gm(observations, visible_params)
        visible_params = maximize_visible_params_gm(observations, hidden_params)
    return visible_params, hidden_params

In [218]:
def generate_norm(params, size, indexes):
    return np.array([stats.norm.rvs(*params[i], size) for i in indexes])

print(generate_norm(np.array([[-10, 0.5], [30, 15]]), 10, [1, 0, 0, 1, 0]))

[[ 22.73138631  38.68844962  31.08984531  49.72170174  15.10917116
   27.09386767  -4.64964849  31.67360694  25.95483314  29.25336147]
 [-10.6357767   -8.97528692  -9.36660848 -10.05024844  -9.72180614
  -10.53131243  -8.82668543 -11.68404029  -9.16825313  -9.61380038]
 [-10.09427618 -10.13917496  -9.74683926 -10.63816671  -9.72704352
  -10.23403944  -9.22729174 -10.31016058  -9.45440579 -10.8207077 ]
 [ 18.45307559  21.00533734  37.44963202  28.90126467  44.80198507
   47.77182779  29.44311313  40.87891534  46.57039318  16.96715578]
 [-10.42086735 -10.54406874  -9.29479122  -9.25389654 -10.25204651
  -10.58037044  -9.71916536  -9.78374837 -10.13441805 -10.44336988]]


In [221]:
expectation_maximization_gm(
    generate_norm(np.array([[-1, 2], [3, 1.5]]), 50, [1, 0, 0, 1, 0]),
    np.array([[1.0, 1.0], [-1.0, 1.0]]),
    10
)

(array([[ 2.97321562,  0.99401806],
        [-1.76863526,  1.96558517]]),
 array([[  1.00000000e+000,   3.16325599e-180,   4.45726359e-221,
           1.00000000e+000,   1.58790268e-216],
        [  6.10627986e-063,   1.00000000e+000,   1.00000000e+000,
           1.11048813e-058,   1.00000000e+000]]))

In [234]:
expectation_maximization_gm(
    generate_norm(np.array([[1, 1], [3, 5]]), 100, [1, 0, 0, 1, 0, 0, 1, 1, 0, 1]),
    np.array([[1.0, 10.0], [-1.0, 10.0]]),
    100
)

(array([[ 3.45806671,  5.17141297],
        [ 0.9473763 ,  0.98778509]]),
 array([[  1.00000000e+00,   1.81537388e-52,   6.69137843e-55,
           1.00000000e+00,   9.77532111e-59,   1.55593867e-60,
           1.00000000e+00,   1.00000000e+00,   1.15272900e-56,
           1.00000000e+00],
        [  0.00000000e+00,   1.00000000e+00,   1.00000000e+00,
           0.00000000e+00,   1.00000000e+00,   1.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
           0.00000000e+00]]))